# Packaging & sharing functions 

So you've created several functions to help you complete analysis on a dataset. But you know that you'll need to use these functions again in the future, and you know the work you've done will be helpful to other colleagues doing similar work. How can you package your functions up in a way that they 'stand alone'? We want them to be easily shareable, readable, and well commented so that any user can utilize them (including you, months from now when you've forgotten the details of the project). We can do this by utilizing **self documentation** in our code base! 

[This](https://realpython.com/documenting-python-code/) is a great article that goes into detail on the importance of documentation & different methods. We will go over some key strategies in this practice. 

Let's bring in some data, create some functions, and document & package them up. 

## Loading in our dataset 

We will read in `ikea.pkl`, which is a **dictionary** containing furniture item available at IKEA & corresponding attributes, edited & reformatted from [this source](https://www.kaggle.com/datasets/ahmedkallam/ikea-sa-furniture-web-scraping?resource=download).  

Recall that a dictionary is a combination of key:value pairs. In this case our "key" is the name of each furniture item, and the "value" is a list containing a few attributes. The attributes are given in the following order: `'category', 'price', 'designer'`.

Note that this is a .pkl file, or a **pickle** file, which allows us to save python data objects in their original formats (e.g. for example you could save a pandas dataframe as is, instead of having to convert it to a csv). To do so you need to import the `pickle` module, and read the data in with the following syntax: 

    ikea = pickle.load(open('ikea.pkl', "rb"))

Where `ikea.pkl` is the file name, and `"rb"` indicates that we are reading a file from binary. 

Let's read in `ikea.pkl` and save it to a variable named `ikea_dict`, then take a peek at the contents.

In [1]:
# read data in 
import pickle
ikea_dict = pickle.load(open('ikea.pkl', "rb"))

# print 
ikea_dict

{'LIXHULT': ['Cabinets & cupboards', 145.0, 'Jon Karlsson'],
 'PLATSA': ['Beds', 2163.5, 'IKEA of Sweden'],
 'BAGGANÄS': ['Bookcases & shelving units', 27.0, 'H Preutz/A Fredriksson'],
 'BESTÅ': ['Bookcases & shelving units', 150.0, 'IKEA of Sweden'],
 'BRENNÅSEN': ['Beds', 30.0, 'Ola Wihlborg'],
 'FANBYN': ['Chairs', 295.0, 'IKEA of Sweden'],
 'BILLY': ['Bookcases & shelving units',
  795.0,
  'IKEA of Sweden/Gillis Lundgren'],
 'FRANKLIN': ['Bar furniture', 129.0, 'K Hagberg/M Hagberg'],
 'LAUVIK': ['Beds', 2425.0, 'Synnöve Mork/Ola Wihlborg/IKEA of Sweden'],
 'MALM': ['Beds', 1595.0, 'Eva Lilja Löwenhielm'],
 'ALGOT': ['Bookcases & shelving units', 388.6, 'Francis Cayouette'],
 'BRUSALI': ['Cabinets & cupboards', 595.0, 'IKEA of Sweden'],
 'BLECKBERGET': ['Chairs', 250.0, 'Francis Cayouette'],
 'EDVALLA': ['Cabinets & cupboards', 25.0, 'Francis Cayouette'],
 'BILLY / OXBERG': ['Cabinets & cupboards', 915.0, 'Gillis Lundgren'],
 'GALANT': ['Cabinets & cupboards', 945.0, 'Eva Lilja Lö

We *cannot* use an index to call a particular item! A dictionary is inherently *un-ordered*. 

We can create a list of all the product names, the "keys", like so, so we have a list of items we may want to query. 

In [2]:
ikea_names = list(ikea_dict.keys())

#print first 10 keys 
print(ikea_names[:10])

# use a key to get its value 
ikea_dict['PLATSA']

['LIXHULT', 'PLATSA', 'BAGGANÄS', 'BESTÅ', 'BRENNÅSEN', 'FANBYN', 'BILLY', 'FRANKLIN', 'LAUVIK', 'MALM']


['Beds', 2163.5, 'IKEA of Sweden']

## Creating a set of functions to work with our data

Say we are working for an interior design firm and our client has requested we select cabinets for an upcoming remodeling project. 

Let's create a function we can use to isolate all items in our dictionary which are of the category 'Cabinets & cupboards'. Since the list attributes are given in order `['category', 'price', 'designer']`, we would want to use the first (0th) item of the list to help us filter. 

The following `find_items` function will help us navigate our dictionary given the input arguments `ikea_dict`, which is our dictionary, and `item_category`, which is the type of furniture we want to filter for. The goal is to return a list of IKEA product names that we can return to our client. We will accomplish this with a few steps: 

1. Create two empty lists we can fill with keys & values of items that meet our conditions 
2. Iterate through each key in `ikea_dict` to & finding the corresponding values 
3. Checking if 'Cabinets & cupboards' is in the first value which contains the category 
4. If true then append this to our `selected_keys` list & `selected_vals` list 
5. Use `zip` to combine the two lists & use `dict` to convert them (full explanation [here](https://appdividend.com/2022/03/10/python-zip-dictionary/). 

In [3]:
def find_products(catalog, product_type):
    selected_keys = []
    selected_vals = []
    for key in catalog:
        val = catalog[key]
        if product_type in val[0]:
            selected_keys.append(key)
            selected_vals.append(val)
    new_dict = dict(zip(selected_keys, selected_vals))
    return new_dict

In [4]:
# use our function to find all keys whose values contain 'Cabinets & cupboards'
selection = find_products(ikea_dict, 'Cabinets & cupboards')
selection

{'LIXHULT': ['Cabinets & cupboards', 145.0, 'Jon Karlsson'],
 'BRUSALI': ['Cabinets & cupboards', 595.0, 'IKEA of Sweden'],
 'EDVALLA': ['Cabinets & cupboards', 25.0, 'Francis Cayouette'],
 'BILLY / OXBERG': ['Cabinets & cupboards', 915.0, 'Gillis Lundgren'],
 'GALANT': ['Cabinets & cupboards', 945.0, 'Eva Lilja Löwenhielm'],
 'DETOLF': ['Cabinets & cupboards', 295.0, 'IKEA of Sweden'],
 'UPPLEVA': ['Cabinets & cupboards', 145.0, 'IKEA of Sweden'],
 'ROTHULT': ['Cabinets & cupboards', 95.0, 'IKEA of Sweden'],
 'BRIMNES': ['Cabinets & cupboards', 952.0, 'K Hagberg/M Hagberg'],
 'HAVSTA': ['Cabinets & cupboards', 4770.0, 'IKEA of Sweden'],
 'SINDVIK': ['Cabinets & cupboards', 125.0, 'Marcus Arvonen'],
 'HÄLLAN': ['Cabinets & cupboards', 1400.0, 'Jon Karlsson'],
 'BEKANT': ['Cabinets & cupboards', 944.0, 'Eva Lilja Löwenhielm'],
 'HACKÅS': ['Cabinets & cupboards', 50.0, 'J Löfgren/J Pettersson']}

Now we have a dictionary of potential cabinets that fit our specifications! However, due to a dispute between the client and the Löwenhielm family, the client has requested that we not include any of Eva Lilja Löwenhielm's designs.  

Let's use this information to narrow down our list. 

We don't want to modify our original funciton since we want to be able to use it for other queries, so let's nest it within a new function named `remove_designer`. We will use **dictionary comprehension** (which uses a very similar syntax to list comprehension) to keep all items in the new dictionary that do **not** contain Eva Lilja Löwenhielm's designs. 

In [5]:
def remove_designer(catalog, product_type, designer_to_exclude):
    selection = find_products(catalog, product_type)
    refined_selection = {key: val for key, val in selection.items() if designer_to_exclude not in val}
    return refined_selection

In [6]:
refined_selection = remove_designer(ikea_dict, 'Cabinets & cupboards', 'Eva Lilja Löwenhielm')
refined_selection

{'LIXHULT': ['Cabinets & cupboards', 145.0, 'Jon Karlsson'],
 'BRUSALI': ['Cabinets & cupboards', 595.0, 'IKEA of Sweden'],
 'EDVALLA': ['Cabinets & cupboards', 25.0, 'Francis Cayouette'],
 'BILLY / OXBERG': ['Cabinets & cupboards', 915.0, 'Gillis Lundgren'],
 'DETOLF': ['Cabinets & cupboards', 295.0, 'IKEA of Sweden'],
 'UPPLEVA': ['Cabinets & cupboards', 145.0, 'IKEA of Sweden'],
 'ROTHULT': ['Cabinets & cupboards', 95.0, 'IKEA of Sweden'],
 'BRIMNES': ['Cabinets & cupboards', 952.0, 'K Hagberg/M Hagberg'],
 'HAVSTA': ['Cabinets & cupboards', 4770.0, 'IKEA of Sweden'],
 'SINDVIK': ['Cabinets & cupboards', 125.0, 'Marcus Arvonen'],
 'HÄLLAN': ['Cabinets & cupboards', 1400.0, 'Jon Karlsson'],
 'HACKÅS': ['Cabinets & cupboards', 50.0, 'J Löfgren/J Pettersson']}

Taking a look at our new dictionary shows that her designs are removed! We can double check this by computing length and making sure those two entries are gone. 

In [7]:
print(len(selection))
print(len(refined_selection))

14
12


<hr style="border:2px solid gray"> </hr>

### Now you try! 

What if the client wants to make sure the design was from a specific designer? Create a function named `select_designer` that takes input arguments `catalog`, `product_type`, and `designer_to_select`. Have it return a `refined_selection` that includes **only** designs from the input designer name. 

Use the function to print all `Cabinets & cupboards` designed in-house by `IKEA of Sweden`. 

In [8]:
### BEGIN SOLUTION 

def select_designer(catalog, product_type, designer_to_select):
    selection = find_products(catalog, product_type)
    refined_selection = {key: val for key, val in selection.items() if designer_to_select in val}
    return refined_selection

select_designer(ikea_dict, 'Cabinets & cupboards', 'IKEA of Sweden')

### END SOLUTION 

{'BRUSALI': ['Cabinets & cupboards', 595.0, 'IKEA of Sweden'],
 'DETOLF': ['Cabinets & cupboards', 295.0, 'IKEA of Sweden'],
 'UPPLEVA': ['Cabinets & cupboards', 145.0, 'IKEA of Sweden'],
 'ROTHULT': ['Cabinets & cupboards', 95.0, 'IKEA of Sweden'],
 'HAVSTA': ['Cabinets & cupboards', 4770.0, 'IKEA of Sweden']}

<hr style="border:2px solid gray"> </hr>

Finally, we want to be able to filter by pricepoint. Our client will let us know their budget so we want to be able to filter our `refined_selection` to account for this. We use a for loop to pass through each key in our list of keys, then use a conditional to see if the second value in the list (the price) is within the client's budget. If it is, add that to our `refined_selection` dictionary to be returned. 

In [18]:
def filter_pricepoint(input_selection, max_price):
    final_selection_keys = []
    final_selection_vals = []
    for key in list(input_selection.keys()):
        val = input_selection[key]
        if val[1] < max_price:
            final_selection_keys.append(key)           
            final_selection_vals.append(val)
    final_selection = dict(zip(final_selection_keys, final_selection_vals))
    return final_selection

final_selection = filter_pricepoint(refined_selection, max_price = 600)
final_selection

{'LIXHULT': ['Cabinets & cupboards', 145.0, 'Jon Karlsson'],
 'BRUSALI': ['Cabinets & cupboards', 595.0, 'IKEA of Sweden'],
 'EDVALLA': ['Cabinets & cupboards', 25.0, 'Francis Cayouette'],
 'DETOLF': ['Cabinets & cupboards', 295.0, 'IKEA of Sweden'],
 'UPPLEVA': ['Cabinets & cupboards', 145.0, 'IKEA of Sweden'],
 'ROTHULT': ['Cabinets & cupboards', 95.0, 'IKEA of Sweden'],
 'SINDVIK': ['Cabinets & cupboards', 125.0, 'Marcus Arvonen'],
 'HACKÅS': ['Cabinets & cupboards', 50.0, 'J Löfgren/J Pettersson']}

## Adding docstrings to our functions 

**Docstrings** are strings built into your functions that can be queried using the built in `help()` function. We can print the docstrings for one of python's built in functions, `enumerate`, like so:

In [ ]:
help(enumerate)

You'll see a few key features such as a summary of the function's use case, describing the input type/description for the function to work properly, and describing the expected outputs. 

We can implement these docstrings in our user-defined functions as well, making sure to include the same key properties. This is done using a string in triple quotation marks `"""` in the first line of the function. Let's add docstrings to the 3 functions we made earlier. We will also add some comments within the function to indicate what is happening at each step. You don't need to comment every single line, but its a good idea to comment out blocks that accomplish certain sub-steps. 

In [23]:
def find_products(catalog, product_type):
    '''
    | Help on user-defined function find_products:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | 
    | This function returns:
    | - new_dict - a dictionary containing only catalog items that match the input product_type
    '''
    # empty list to fill with items 
    selected_keys = []
    selected_vals = []
    # loop through each key 
    for key in catalog:
    # for each key, find the values 
        val = catalog[key]
        # check if the item category is in the values
        # if it is, add it to our selection list
        if product_type in val[0]:
            selected_keys.append(key)
            selected_vals.append(val)
    # make new dict to return 
    new_dict = dict(zip(selected_keys, selected_vals))
    # return our list 
    return new_dict

def remove_designer(catalog, product_type, designer_to_exclude):
    '''
    | Help on user-defined function remove_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_exclude - string containing name of designer we want to remove from selection 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude
    '''
    # run find_products, nested function 
    selection = find_products(catalog, product_type)
    # exclude designer in selection
    refined_selection = {key: val for key, val in selection.items() if designer_to_exclude not in val}
    return refined_selection

def filter_pricepoint(input_selection, max_price):
    '''
    | Help on user-defined function filter_pricepoint:
    |
    | This function takes inputs: 
    | - input_selection - dictionary containing ikea catalog items. can be original catalog dictionary or
    | filtered `refined_selection` dict from find_products, remove_designer, or other. 
    | - max_price - int or float indicating max budget. funciton will keep everything < this value. 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude
    '''
    # empty lists to fill with keys & vals we keep 
    final_selection_keys = []
    final_selection_vals = []
    # loop through keys, get vals 
    for key in list(input_selection.keys()):
        val = input_selection[key]
        # if within budget add keys & vals to list
        if val[1] < max_price:
            input_selection_keys.append(key)           
            input_selection_vals.append(val)
    # zip lists into a new dictionary 
    final_selection = dict(zip(final_selection_keys, final_selection_vals))
    return final_selection

In [24]:
help(find_products)

Help on function find_products in module __main__:

find_products(catalog, product_type)
    | Help on user-defined function find_products:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | 
    | This function returns:
    | - new_dict - a dictionary containing only catalog items that match the input product_type



In [25]:
help(remove_designer)

Help on function remove_designer in module __main__:

remove_designer(catalog, product_type, designer_to_exclude)
    | Help on user-defined function remove_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_exclude - string containing name of designer we want to remove from selection 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude



In [26]:
help(filter_pricepoint)

Help on function filter_pricepoint in module __main__:

filter_pricepoint(input_selection, max_price)
    | Help on user-defined function filter_pricepoint:
    |
    | This function takes inputs: 
    | - input_selection - dictionary containing ikea catalog items. can be original catalog dictionary or
    | filtered `refined_selection` dict from find_products, remove_designer, or other. 
    | - max_price - int or float indicating max budget. funciton will keep everything < this value. 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude



<hr style="border:2px solid gray"> </hr>

### Now you try! 

Add docstrings for the `select_designer` function you created earlier. Print the docstring using `help()`. 

In [30]:
### BEGIN SOLUTION 

def select_designer(catalog, product_type, designer_to_select):
    '''    
    | Help on user-defined function select_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_select - string containing name of designer we want to keep (exclude every other designer)
    | 
    | This function returns:
    | - refined_selection - a dictionary containing only catalog items that match the input product_type 
    | AND is designed by designer_to_select
    '''
    selection = find_products(catalog, product_type)
    refined_selection = {key: val for key, val in selection.items() if designer_to_select in val}
    return refined_selection

help(select_designer)

### END SOLUTION 

Help on function select_designer in module __main__:

select_designer(catalog, product_type, designer_to_select)
    | Help on user-defined function select_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_select - string containing name of designer we want to keep (exclude every other designer)
    | 
    | This function returns:
    | - refined_selection - a dictionary containing only catalog items that match the input product_type 
    | AND is designed by designer_to_select



<hr style="border:2px solid gray"> </hr>

## Packaging a set of functions into a module

A **module** is a set of definitions/functions pacakged into a script that can be called into any workspace. This is simply a `.py` file that contains our function definitions. For example a module called `ikea_functions.py` could be called in using `import ikea_functions`. The syntax is the same as when we import other packages such as pandas or matplotlib. 

Let's create a module called `ikea_functions` that contains all the functions we defined. You can do this in a text editor where you simply name the file `ikea_functions.py` or you can use the method below to write a file directly from jupyter:

In [42]:
%%writefile ikea_functions.py

def find_products(catalog, product_type):
    '''
    | Help on user-defined function find_products:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | 
    | This function returns:
    | - new_dict - a dictionary containing only catalog items that match the input product_type
    '''
    # empty list to fill with items 
    selected_keys = []
    selected_vals = []
    # loop through each key 
    for key in catalog:
    # for each key, find the values 
        val = catalog[key]
        # check if the item category is in the values
        # if it is, add it to our selection list
        if product_type in val[0]:
            selected_keys.append(key)
            selected_vals.append(val)
    # make new dict to return 
    new_dict = dict(zip(selected_keys, selected_vals))
    # return our list 
    return new_dict

def remove_designer(catalog, product_type, designer_to_exclude):
    '''
    | Help on user-defined function remove_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_exclude - string containing name of designer we want to remove from selection 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude
    '''
    # run find_products, nested function 
    selection = find_products(catalog, product_type)
    # exclude designer in selection
    refined_selection = {key: val for key, val in selection.items() if designer_to_exclude not in val}
    return refined_selection

def filter_pricepoint(input_selection, max_price):
    '''
    | Help on user-defined function filter_pricepoint:
    |
    | This function takes inputs: 
    | - input_selection - dictionary containing ikea catalog items. can be original catalog dictionary or
    | filtered `refined_selection` dict from find_products, remove_designer, or other. 
    | - max_price - int or float indicating max budget. funciton will keep everything < this value. 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude
    '''
    # empty lists to fill with keys & vals we keep 
    final_selection_keys = []
    final_selection_vals = []
    # loop through keys, get vals 
    for key in list(input_selection.keys()):
        val = input_selection[key]
        # if within budget add keys & vals to list
        if val[1] < max_price:
            input_selection_keys.append(key)           
            input_selection_vals.append(val)
    # zip lists into a new dictionary 
    final_selection = dict(zip(final_selection_keys, final_selection_vals))
    return final_selection

def select_designer(catalog, product_type, designer_to_select):
    '''    
    | Help on user-defined function select_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_select - string containing name of designer we want to keep (exclude every other designer)
    | 
    | This function returns:
    | - refined_selection - a dictionary containing only catalog items that match the input product_type 
    | AND is designed by designer_to_select
    '''
    selection = find_products(catalog, product_type)
    refined_selection = {key: val for key, val in selection.items() if designer_to_select in val}
    return refined_selection



Overwriting ikea_functions.py


Let's clear our variable space, import our module using `import ikea_functions` & re-load our ikea file to test this. 

In [55]:
# reset variables space 
# run this then type 'y' when prompted
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [59]:
# reload file
import pickle
ikea_dict = pickle.load(open('ikea.pkl', "rb"))

# import the entire module 
import ikea_functions

# and use dir to print all functions within the module 
dir(ikea_functions)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'filter_pricepoint',
 'find_products',
 'remove_designer',
 'select_designer']

You see above that our four functions are listed within the new `ikea_functions` we imported! We can access a particular function using dot notation like this: 

In [60]:
ikea_functions.find_products(ikea_dict, 'Beds')

{'PLATSA': ['Beds', 2163.5, 'IKEA of Sweden'],
 'BRENNÅSEN': ['Beds', 30.0, 'Ola Wihlborg'],
 'LAUVIK': ['Beds', 2425.0, 'Synnöve Mork/Ola Wihlborg/IKEA of Sweden'],
 'MALM': ['Beds', 1595.0, 'Eva Lilja Löwenhielm'],
 'LYCKSELE HÅVET': ['Beds', 795.0, 'IKEA of Sweden'],
 'SULTAN': ['Beds', 50.0, 'IKEA of Sweden'],
 'DELAKTIG': ['Beds', 200.0, 'Tom Dixon'],
 'VITVAL': ['Beds', 295.0, 'Tord Björklund'],
 'LIDHULT': ['Beds', 5553.0, 'IKEA of Sweden/Henrik Preutz'],
 'TUFFING': ['Beds', 645.0, 'IKEA of Sweden'],
 'VIKARE': ['Beds', 95.0, 'IKEA of Sweden'],
 'GULLIVER': ['Beds', 595.0, 'IKEA of Sweden']}

We could also import only a single function from the module & use it, like so: 

In [61]:
from ikea_functions import find_products

find_products(ikea_dict, 'Beds')

{'PLATSA': ['Beds', 2163.5, 'IKEA of Sweden'],
 'BRENNÅSEN': ['Beds', 30.0, 'Ola Wihlborg'],
 'LAUVIK': ['Beds', 2425.0, 'Synnöve Mork/Ola Wihlborg/IKEA of Sweden'],
 'MALM': ['Beds', 1595.0, 'Eva Lilja Löwenhielm'],
 'LYCKSELE HÅVET': ['Beds', 795.0, 'IKEA of Sweden'],
 'SULTAN': ['Beds', 50.0, 'IKEA of Sweden'],
 'DELAKTIG': ['Beds', 200.0, 'Tom Dixon'],
 'VITVAL': ['Beds', 295.0, 'Tord Björklund'],
 'LIDHULT': ['Beds', 5553.0, 'IKEA of Sweden/Henrik Preutz'],
 'TUFFING': ['Beds', 645.0, 'IKEA of Sweden'],
 'VIKARE': ['Beds', 95.0, 'IKEA of Sweden'],
 'GULLIVER': ['Beds', 595.0, 'IKEA of Sweden']}

See that our docstrings can be printed too! 

In [62]:
help(ikea_functions.remove_designer)

Help on function remove_designer in module ikea_functions:

remove_designer(catalog, product_type, designer_to_exclude)
    | Help on user-defined function remove_designer:
    |
    | This function takes inputs: 
    | - catalog - dictionary containing ikea catalog items
    | - product_type - string containing product types in catalog (first item in dict value)
    | - designer_to_exclude - string containing name of designer we want to remove from selection 
    | 
    | This function returns:
    | - final_selection - a dictionary containing only catalog items that match the input product_type 
    | AND does NOT include designer_to_exclude



Importing a module is a lot less cumbersome than redefining the function in every script you want to use it. This makes it easy to re-use functions in multiple scripts. If you make a change to a function in the module, it will carry through to every usage of the function thereafter, whereas if you were to have the function defined in multiple scripts it is likely you will forget to change every instance and the "same" function will behave differently in multiple scripts. 

It also makes it more easily shareable, as you can simply send one .py file to a colleage and they can import the functions you made. This way they will not have to hard code them themselves or copy/paste from a different script. The use of docstrings in your module functions allows you or a colleague to easily access documentation so you don't need to re-inspect the entire funtion to remember inputs, formats, etc. 

# Practice on your own 

